In [2]:
using Gurobi
using CSV
using DataFrames
using JuMP

┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\Gurobi\do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190


In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\steph\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [3]:
df = CSV.read("ev_constraints.csv", header = true, missingstring="\n")

,station_1,station_2,station_3,station_4,station_5,station_6,station_7,station_8,station_9
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,5,missing,missing,missing,missing,missing,missing,missing
2,5,missing,missing,missing,missing,missing,missing,missing,missing
3,2,3,4,18,missing,missing,missing,missing,missing
4,3,4,18,missing,missing,missing,missing,missing,missing
5,4,18,missing,missing,missing,missing,missing,missing,missing
6,2,3,4,18,missing,missing,missing,missing,missing
7,3,4,18,missing,missing,missing,missing,missing,missing
8,2,3,4,15,missing,missing,missing,missing,missing
9,2,3,4,15,missing,missing,missing,missing,missing


In [6]:
num_nodes = 50

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:num_nodes],Bin)

for row in eachrow(df)
    v = vec(convert(Array, row)) # no transpose necessary
    @constraint(m, sum(x[i] for i in v if !ismissing(i)) >= 1)
end

@objective(m, Min, sum(x[i] for i in 1:num_nodes))


┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[6]:8
└ @ Core .\In[6]:8


BoundsError: BoundsError: attempt to access 50-element Array{Variable,1} at index [0]

In [18]:
status = solve(m)

Academic license - for non-commercial use only


:Optimal

In [15]:
Pkg.rm("Gurobi")
Pkg.add("Gurobi")
Pkg.build("Gurobi")

  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Project.toml`
  [2e9cd046] - Gurobi v0.6.0
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Manifest.toml`
  [2e9cd046] - Gurobi v0.6.0
 Resolving package versions...
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Project.toml`
  [2e9cd046] + Gurobi v0.6.0
  Updating `C:\Users\steph\Downloads\15.053_2019SP-master (2)\15.053_2019SP-master\Manifest.toml`
  [2e9cd046] + Gurobi v0.6.0
  Building Gurobi → `C:\Users\steph\.julia\packages\Gurobi\dlJep\deps\build.log`


In [19]:
getvalue(x)

18-element Array{Float64,1}:
  0.0
  1.0
  1.0
  0.0
  0.0
  0.0
  0.0
 -0.0
  1.0
  0.0
  1.0
  0.0
  0.0
  1.0
 -0.0
  0.0
  0.0
  0.0

,station_1,station_2,station_3
1,2,missing,missing


In [64]:
 ismissing(df[1,1])

false

In [68]:
m

Feasibility problem with:
 * 114 linear constraints
 * 18 variables: 18 binary
Solver is Gurobi